In [1]:
import xgboost as xgb
import pandas as pd
import model_helper_functions as mod
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error,  r2_score
from sklearn.preprocessing import LabelEncoder

In [10]:
df =  pd.read_parquet("./all_cleaned_data/all_cleaned_data.parquet")
df.head()

,VendorID,PickupDatetime,DropoffDatetime,TripDuration,PassengerCount,TripDistance,PULocationID,DOLocationID,PaymentType,FareAmount,ExtraCharges,MTATax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
0,2,2024-01-01 00:46:55,2024-01-01 00:58:25,11.500000,1.0,1.98,236,239,1.0,12.8,1.0,0.5,3.61,0.0,1.0,21.66,2.75,0
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,20.866667,5.0,6.54,65,170,1.0,30.3,1.0,0.5,7.11,0.0,1.0,42.66,2.75,0
2,2,2024-01-01 00:30:21,2024-01-01 00:49:23,19.033333,1.0,3.08,74,262,1.0,19.8,1.0,0.5,3.00,0.0,1.0,28.05,2.75,0
3,1,2024-01-01 00:30:20,2024-01-01 00:42:12,11.866667,1.0,2.40,74,116,2.0,14.2,1.0,1.5,0.00,0.0,1.0,16.70,0.00,0
4,2,2024-01-01 00:32:38,2024-01-01 00:43:37,10.983333,1.0,5.14,74,243,1.0,22.6,1.0,0.5,6.28,0.0,1.0,31.38,0.00,0


In [11]:
df = mod.necessary_fields(df)

df.head()

,PickupDatetime,DropoffDatetime,TripDuration,TripDistance,PULocationID,DOLocationID,FareAmount,TipAmount
0,2024-01-01 00:46:55,2024-01-01 00:58:25,11.500000,1.98,236,239,12.8,3.61
1,2024-01-01 00:31:42,2024-01-01 00:52:34,20.866667,6.54,65,170,30.3,7.11
2,2024-01-01 00:30:21,2024-01-01 00:49:23,19.033333,3.08,74,262,19.8,3.00
3,2024-01-01 00:30:20,2024-01-01 00:42:12,11.866667,2.40,74,116,14.2,0.00
4,2024-01-01 00:32:38,2024-01-01 00:43:37,10.983333,5.14,74,243,22.6,6.28


In [12]:
df['Hour'] = df['DropoffDatetime'].apply(mod.round_time_to_int)
df.head()

,PickupDatetime,DropoffDatetime,TripDuration,TripDistance,PULocationID,DOLocationID,FareAmount,TipAmount,Hour
0,2024-01-01 00:46:55,2024-01-01 00:58:25,11.500000,1.98,236,239,12.8,3.61,1
1,2024-01-01 00:31:42,2024-01-01 00:52:34,20.866667,6.54,65,170,30.3,7.11,1
2,2024-01-01 00:30:21,2024-01-01 00:49:23,19.033333,3.08,74,262,19.8,3.00,1
3,2024-01-01 00:30:20,2024-01-01 00:42:12,11.866667,2.40,74,116,14.2,0.00,1
4,2024-01-01 00:32:38,2024-01-01 00:43:37,10.983333,5.14,74,243,22.6,6.28,1


In [13]:
df = df.drop(columns = ['PickupDatetime', 'DropoffDatetime', 'TripDuration', 'TripDistance', 'FareAmount', 'TipAmount'])

df.head()


,PULocationID,DOLocationID,Hour
0,236,239,1
1,65,170,1
2,74,262,1
3,74,116,1
4,74,243,1


In [15]:
X,y = df[['DOLocationID', 'Hour']], df['PULocationID']
del df

le = LabelEncoder()
y = le.fit_transform(y)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
model = xgb.XGBRegressor(n_estimators = 1000, max_depth = 4, eta = 0.1, subsample = 0.1, colsample_bytree = 1.0) # Need to fine tune all these parameters
model.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [9]:
# Training performance:
pred_y = model.predict(train_X)

print("Train R2-score:", r2_score(train_y, pred_y))
print("Train RMSE:", root_mean_squared_error(train_y, pred_y))

Train R2-score: 0.039433956146240234
Train RMSE: 62.71317585405906


In [8]:
# Testing performance:
pred_y = model.predict(test_X)

print("Test R2-score:", r2_score(test_y, pred_y))
print("Test RMSE:", root_mean_squared_error(test_y, pred_y))

Test R2-score: 0.03961306810379028
Test RMSE: 62.69765896096037


In [11]:
# Saving Model:
model.save_model("./xgboost_models/xgboost_test_save.json")

In [6]:
model = xgb.XGBClassifier(n_estimators = 100, max_depth = 2, eta = 0.1, subsample = 0.1)
model.fit(train_X, train_y)

NameError: name 'train_X' is not defined

In [ ]:
# Training performance:
pred_y = model.predict(train_X)

print("Train R2-score:", r2_score(train_y, pred_y))
print("Train RMSE:", root_mean_squared_error(train_y, pred_y))

In [ ]:
# Testing performance:
pred_y = model.predict(test_X)

print("Test R2-score:", r2_score(test_y, pred_y))
print("Test RMSE:", root_mean_squared_error(test_y, pred_y))

In [ ]:
# Saving Model:
model.save_model("./xgboost_models/xgboost_test_classifier.json")